In [ ]:
import numpy as np
import tensorflow as tf
import os

# GPUを使用しないように設定（CPUで実行する場合）
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# TensorFlow 1.x系のグラフをリセット
tf.reset_default_graph()

# --- 設定項目 (ご自身の環境に合わせてパスを修正してください) ---

# 1. 推論したい未知のデータファイルのパス
X_VAL_PATH = r"C:\\path\\to\\your\\X_val.txt" 

# 2. 学習済みモデルが保存されているパス
MODEL_PATH = r"C:\\Users\\admin\\Downloads\\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\\weights\\model.ckpt"

# 3. モデルの基本設定 (学習時と完全に同じにする必要があります)
LABELS = [
    "ageru",
    "understand",
    "annsinnsuru",
    "heavy"
]
n_steps = 30
n_hidden = 30
n_classes = 4

# --- データ読み込み関数 ---
def load_X(X_path):
    """
    指定されたパスからデータを読み込み、モデルの入力形式に整形します。
    """
    try:
        with open(X_path, 'r') as file:
            X_ = np.array(
                [elem for elem in [
                    row.split(',') for row in file
                ]],
                dtype=np.float32
            )
        blocks = int(len(X_) / n_steps)
        X_ = np.array(np.split(X_, blocks))
        return X_
    except FileNotFoundError:
        print(f"エラー: ファイルが見つかりません - {X_path}")
        return None
    except Exception as e:
        print(f"データ読み込み中にエラーが発生しました: {e}")
        return None

# --- LSTMモデルの定義 (学習時と同じ構造) ---
def LSTM_RNN(_X, _weights, _biases, n_input):
    _X = tf.transpose(_X, [1, 0, 2])
    _X = tf.reshape(_X, [-1, n_input])
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    _X = tf.split(_X, n_steps, 0)
    
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    lstm_last_output = outputs[-1]
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']

# --- メインの処理 ---
if __name__ == '__main__':
    # --- 未知データを読み込む ---
    print(f"推論データを読み込んでいます: {X_VAL_PATH}")
    X_val = load_X(X_VAL_PATH)

    if X_val is not None:
        n_input = len(X_val[0][0])
        
        # --- TensorFlowグラフ構築 ---
        x = tf.placeholder(tf.float32, [None, n_steps, n_input])
        
        weights = {
            'hidden': tf.Variable(tf.zeros([n_input, n_hidden]), name="weights_hidden"),
            'out': tf.Variable(tf.zeros([n_hidden, n_classes]), name="weights_out")
        }
        biases = {
            'hidden': tf.Variable(tf.zeros([n_hidden]), name="biases_hidden"),
            'out': tf.Variable(tf.zeros([n_classes]), name="biases_out")
        }

        # 予測オペレーションを定義
        pred = LSTM_RNN(x, weights, biases, n_input)
        
        # モデルを保存・復元するためのSaver
        saver = tf.train.Saver()
        
        # --- セッションを開始し、モデルを読み込んで推論を実行 ---
        with tf.Session() as sess:
            try:
                # 学習済みモデルの重みを復元
                saver.restore(sess, MODEL_PATH)
                print(f"モデルを復元しました: {MODEL_PATH}")

                # 推論を実行
                one_hot_predictions = sess.run(
                    [pred],
                    feed_dict={x: X_val}
                )
                
                # 最も確率の高いクラスのインデックスを取得
                predictions = one_hot_predictions[0].argmax(1)

                # --- 推論結果を出力 ---
                print("\\n--- 推論結果 ---")
                for i, predicted_label_index in enumerate(predictions):
                    predicted_label_name = LABELS[predicted_label_index]
                    print(
                        f"データ #{i + 1:03d}: 予測='{predicted_label_name}'"
                    )

            except Exception as e:
                print(f"モデルの復元または推論中にエラーが発生しました: {e}")
                print("MODEL_PATHが正しいか、学習済みモデルファイルが存在するか確認してください。")